# Where to Open a Pizza Restaurant in the North/Northwets Suburbs of Detroit

### Obtain necessary packages and import statements

In [1]:
!conda install -c conda-forge geocoder -y
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge scikit-learn -y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTAL

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import geocoder
print('Ready!')

Ready!


### Data

urls (names of suburbs, counties)

In [3]:
# List of Detroit's North/Northwest Suburbs 
url_suburbs = 'https://guides.lib.umich.edu/c.php?g=283069&p=1885993'
# Table of Michigan Cities with County and Zip Code
url_counties = 'https://www.zipcodestogo.com/Michigan/'

csv (for lat/long data)

In [4]:
lat_long = pd.read_csv('zipcode.csv')
lat_long.head()

,zip,city,state,latitude,longitude,timezone,dst
0,210,Portsmouth,NH,43.005895,-71.013202,-5,1
1,211,Portsmouth,NH,43.005895,-71.013202,-5,1
2,212,Portsmouth,NH,43.005895,-71.013202,-5,1
3,213,Portsmouth,NH,43.005895,-71.013202,-5,1
4,214,Portsmouth,NH,43.005895,-71.013202,-5,1


### Scrape urls for data

In [5]:
request = requests.get(url_suburbs)
data_suburbs = request.text
soup_suburbs = BeautifulSoup(data_suburbs, 'html.parser')

city_list = []
for tag in soup_suburbs.find_all('a', class_ = ""):
    city_list.append(tag.text)
    
city_list = city_list[7:47]

suburb_city_df = pd.DataFrame(city_list)
suburb_city_df.rename({0:'suburb_name'}, axis = 1, inplace = True)
suburb_city_df.head()

,suburb_name
0,Auburn Hills
1,Berkley
2,Beverly Hills
3,Birmingham
4,Bloomfield Hills


In [6]:
request_2 = requests.get(url_counties)
data_counties = request_2.text
soup_counties = BeautifulSoup(data_counties, 'html.parser')
table_counties = soup_counties.table
table_rows = table_counties.find_all('tr')

table_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    table_list.append(row)
    
county_df = pd.DataFrame(table_list[3:(len(table_list))])
county_df.drop(3, axis = 1, inplace = True)
county_df.rename({0:'zip', 1: 'suburb_name', 2: 'county'}, axis = 1, inplace = True)
county_df.head()

,zip,suburb_name,county
0,48001,Algonac,Saint Clair
1,48002,Allenton,Saint Clair
2,48003,Almont,Lapeer
3,48004,Anchorville,Saint Clair
4,48005,Armada,Macomb


In [7]:
df_whole = pd.merge(suburb_city_df, county_df, on = 'suburb_name', how = 'inner')
df_whole['zip'] = df_whole['zip'].astype(int)
print(df_whole.head(), '\n', '\n', 'Size of Merged DataFrame: {}'.format(df_whole.shape))

    suburb_name    zip   county
0  Auburn Hills  48321  Oakland
1  Auburn Hills  48326  Oakland
2       Berkley  48072  Oakland
3    Birmingham  48009  Oakland
4    Birmingham  48012  Oakland 
 
 Size of Merged DataFrame: (67, 3)


DataFrame is larger then I expected. Expected 40 rows (one value for each suburb from suburb_city_df) but it turns out one suburb may have multiple zip codes. This is acceptable and I will move forward using this knowledge.

In [8]:
df_detroit_burbs = pd.merge(df_whole, lat_long, on = 'zip', how = 'inner')
df_detroit_burbs.drop(['city', 'state', 'timezone', 'dst'], axis = 1, inplace = True)
print(df_detroit_burbs.head(), '\n', '\n', 'Size of Merged DataFrame: {}'.format(df_detroit_burbs.shape))

    suburb_name    zip   county   latitude  longitude
0  Auburn Hills  48321  Oakland  42.660091 -83.386300
1  Auburn Hills  48326  Oakland  42.661129 -83.245710
2       Berkley  48072  Oakland  42.497035 -83.185320
3    Birmingham  48009  Oakland  42.544084 -83.215270
4    Birmingham  48012  Oakland  42.604425 -83.292382 
 
 Size of Merged DataFrame: (66, 5)


Lost one zip code due to no lat/long info being available for it. It was an entry for Southfield, MI which had 7 zip codes overall. Due to the large amount of zip codes for this one city, I decided that this one entry could be excluded.

### Visualize Suburb Locations

In [9]:
map_detroit_suburbs = folium.Map(location = (42.33, -83.0458), zoom_start = 9)

for lat, lng, zip1, burb in zip(df_detroit_burbs['latitude'], df_detroit_burbs['longitude'], df_detroit_burbs['zip'], df_detroit_burbs['suburb_name']):
    label = '{}, {}'.format(zip1, burb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#9932cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_detroit_suburbs)  
    
map_detroit_suburbs

After viewing the map, I want to remove 48065 (Romeo) and 48122 (Melvindale) from my data. These are out of the boundaries of what I wanted to look at. Melvindale is not in the north/northwest and Romeo is too far north compared to the other points.

I am also going to move my map center to be a little north of Detroit.

In [11]:
print('Indexes for dropped cities:')
print(df_detroit_burbs.loc[df_detroit_burbs['zip'] == 48065])
print(df_detroit_burbs.loc[df_detroit_burbs['zip'] == 48122])

df_detroit_burbs_2 = df_detroit_burbs.drop([23, 31])
df_detroit_burbs_2.shape

Indexes for dropped cities:
   suburb_name    zip  county   latitude  longitude
31       Romeo  48065  Macomb  42.826805  -83.02963
   suburb_name    zip county   latitude  longitude
23  Melvindale  48122  Wayne  42.281638  -83.18188


(64, 5)

In [12]:
map_detroit_suburbs_2 = folium.Map(location = (42.4895, -83.1446), zoom_start = 10)

for lat, lng, zip1, burb in zip(df_detroit_burbs_2['latitude'], df_detroit_burbs_2['longitude'], df_detroit_burbs_2['zip'], df_detroit_burbs_2['suburb_name']):
    label = '{}, {}'.format(zip1, burb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#9932cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_detroit_suburbs_2)  
    
map_detroit_suburbs_2

### Explore restaurants in the area using Foursquare API

#### Credentials

In [13]:
CLIENT_ID = '2XAVNQNEMFRZHF4QDDUW0V4H0FOH3MR2OBQF24SADGXDXQGY'
CLIENT_SECRET = 'GJZB1SCK3104LPOTD2PH4STWVHT0HXEOCS1BNHHPHIKDN5FT'
VERSION = '20180605'

#### Function to get category of venues

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Function to get nearby venues

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Find nearby venues

In [16]:
LIMIT = 100
detroit_suburb_venues = getNearbyVenues(names = df_detroit_burbs_2['zip'],
                                       latitudes = df_detroit_burbs_2['latitude'],
                                       longitudes = df_detroit_burbs_2['longitude'])
print(detroit_suburb_venues.shape)
#group by venue counts to get an idea of which areas have the most venues
burbs_grouped = detroit_suburb_venues.groupby('Zip').count().sort_values(by = 'Venue', ascending = False)
print(burbs_grouped.shape)
burbs_grouped

(783, 7)
(59, 6)


,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip,,,,,,
48009,80,80,80,80,80,80
48220,61,61,61,61,61,61
48084,53,53,53,53,53,53
48007,37,37,37,37,37,37
48012,32,32,32,32,32,32
48026,27,27,27,27,27,27
48017,25,25,25,25,25,25
48068,23,23,23,23,23,23
48321,23,23,23,23,23,23


#### Find 10 most common venues for each zip code.

In [17]:
burbs_onehot = pd.get_dummies(detroit_suburb_venues[['Venue Category']], prefix="", prefix_sep="")

burbs_onehot['Zip'] = detroit_suburb_venues['Zip'] 

fixed_columns = [burbs_onehot.columns[-1]] + list(burbs_onehot.columns[:-1])
burbs_onehot = burbs_onehot[fixed_columns]

print('Onehot Shape: {}'.format(burbs_onehot.shape))
burbs_groups = burbs_onehot.groupby('Zip').mean().reset_index()
print('Grouped shape: {}'.format(burbs_groups.shape))
burbs_groups.head()

Onehot Shape: (783, 174)
Grouped shape: (59, 174)


,Zip,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,...,Veterinarian,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,48007,0.0,0.027027,0.0,0.0000,0.0,0.027027,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.027027,0.0,0.0,0.000000,0.0000,0.000
1,48009,0.0,0.062500,0.0,0.0125,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,0.0125,0.025
2,48012,0.0,0.031250,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.031250,0.0,0.0,0.000000,0.0000,0.000
3,48017,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.04,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.000
4,48021,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.090909,0.0000,0.000


Function to find most common venues by category name

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Zip']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

burbs_venues_sorted = pd.DataFrame(columns=columns)
burbs_venues_sorted['Zip'] = burbs_groups['Zip']

for ind in np.arange(burbs_groups.shape[0]):
    burbs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(burbs_groups.iloc[ind, :], num_top_venues)

burbs_venues_sorted.head()

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,48007,Furniture / Home Store,Pet Store,Discount Store,Coffee Shop,Paper / Office Supplies Store,Mexican Restaurant,Salon / Barbershop,Fast Food Restaurant,Electronics Store,Breakfast Spot
1,48009,American Restaurant,Spa,Coffee Shop,Steakhouse,Middle Eastern Restaurant,Yoga Studio,Ice Cream Shop,Italian Restaurant,Gym,Men's Store
2,48012,Furniture / Home Store,Sandwich Place,Pizza Place,Deli / Bodega,Shipping Store,Skating Rink,Martial Arts Dojo,Breakfast Spot,Fast Food Restaurant,Salon / Barbershop
3,48017,Pizza Place,Bookstore,Gift Shop,Mexican Restaurant,Café,Chinese Restaurant,Liquor Store,Mobile Phone Shop,Record Shop,Bar
4,48021,Pizza Place,Ice Cream Shop,Bakery,Convenience Store,Coffee Shop,Breakfast Spot,Sushi Restaurant,Liquor Store,Board Shop,Wings Joint


Since I am looking for an area without a lot of pizza places, I want to remove every zip code where 'Pizza Place' is one of the 10 most common venues.

In [20]:
burbs_no_pizza = burbs_venues_sorted.replace('Pizza Place', np.nan)
burbs_no_pizza.dropna(inplace = True)
burbs_no_pizza.head()
burbs_no_pizza.reset_index(drop = True, inplace = True)
print(burbs_no_pizza.shape)
burbs_no_pizza.head()

(46, 11)


,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,48007,Furniture / Home Store,Pet Store,Discount Store,Coffee Shop,Paper / Office Supplies Store,Mexican Restaurant,Salon / Barbershop,Fast Food Restaurant,Electronics Store,Breakfast Spot
1,48009,American Restaurant,Spa,Coffee Shop,Steakhouse,Middle Eastern Restaurant,Yoga Studio,Ice Cream Shop,Italian Restaurant,Gym,Men's Store
2,48030,Sandwich Place,Dry Cleaner,Shipping Store,Middle Eastern Restaurant,Diner,Restaurant,Smoke Shop,Food,Mobile Phone Shop,Park
3,48034,Gym,Home Service,Burger Joint,Business Service,Auto Workshop,Electronics Store,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market
4,48037,Pool,Hotel,Community Center,Gym / Fitness Center,Dive Bar,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant


47 zip codes have no pizza place in the top 10 most common venues.

#### Add lat/long data to zip codes with no pizza places

In [21]:
lat_long_only = df_detroit_burbs[['zip','latitude', 'longitude']]
lat_long_only.rename({'zip':'Zip',}, axis = 1, inplace = True)
no_pizza = burbs_no_pizza.merge(lat_long_only, on = 'Zip', how = 'left')
no_pizza.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,latitude,longitude
0,48007,Furniture / Home Store,Pet Store,Discount Store,Coffee Shop,Paper / Office Supplies Store,Mexican Restaurant,Salon / Barbershop,Fast Food Restaurant,Electronics Store,Breakfast Spot,42.606088,-83.297593
1,48009,American Restaurant,Spa,Coffee Shop,Steakhouse,Middle Eastern Restaurant,Yoga Studio,Ice Cream Shop,Italian Restaurant,Gym,Men's Store,42.544084,-83.215270
2,48030,Sandwich Place,Dry Cleaner,Shipping Store,Middle Eastern Restaurant,Diner,Restaurant,Smoke Shop,Food,Mobile Phone Shop,Park,42.459022,-83.098190
3,48034,Gym,Home Service,Burger Joint,Business Service,Auto Workshop,Electronics Store,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,42.474234,-83.290780
4,48037,Pool,Hotel,Community Center,Gym / Fitness Center,Dive Bar,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,42.567543,-83.149978


#### Function to get individual venue ids

In [22]:
def getNearbyVenuesID(names, latitudes, longitudes, radius=500):
    
    id_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        id_list.append([(
            name, 
            v['venue']['id']) for v in results])

    venue_ids = pd.DataFrame([item for id_list in id_list for item in id_list])
    venue_ids.columns = ['Zip', 'Venue_ID']
    
    return(venue_ids)

#### Function to get individual venue ratings

In [23]:
def getRatings(ids):
    
    ratings_list=[]
    for entry in ids:
        
        try: 
            url_real = 'https://api.foursquare.com/v2/venues/{}'.format(entry)
            params_def = dict(client_id = CLIENT_ID, client_secret = CLIENT_SECRET, v = VERSION)
            result = requests.get(url = url_real, params = params_def).json()["response"]["venue"]["rating"]
            
        except KeyError:
            result = 'NA'

        ratings_list.append([entry, result])

    venue_ratings = pd.DataFrame(ratings_list)
    venue_ratings.columns = ['Venue_ID', 'Rating']
    
    return(venue_ratings)

I can only complete 500 premium requests from the foursquare api a day so I need to split my IDs into two even(-ish) parts. I called the first part, saved it and then ran the second half the day after.

#### Get nearby venue ids

In [ ]:
LIMIT = 100
IDs = getNearbyVenuesID(names = no_pizza['Zip'], latitudes = no_pizza['latitude'], longitudes = no_pizza['longitude'], radius=500)

In [ ]:
IDs.to_pickle("IDs_all")

#### Split IDs data to comply with foursquare daily rate requests.

In [ ]:
IDs_half1 = IDs[:300]
IDs_half2 = IDs[300:]
print(IDs_half1.shape, IDs_half2.shape)

#### Get Ratings (First Half)

In [ ]:
ratings_half1 = getRatings(ids = IDs_half1['Venue_ID'])
print(ratings_half1.shape)
ratings_half1.to_pickle("all_ratings_final")

#### Get Ratings (Second Half)

In [ ]:
ratings_half2 = getRatings(ids = IDs_half2["Venue_ID"])
print(ratings_half2.shape)

#### Combine

In [ ]:
ratings_total = pd.read_pickle('all_ratings_final')
ratings_final = pd.concat([ratings_total, ratings_half2], ignore_index = True)
ratings_final.to_pickle('all_ratings_final')

In [26]:
ratings_final = pd.read_pickle('all_ratings_final')
IDs = pd.read_pickle('IDs_all')

In [27]:
df_detroit_burbs_2.rename({'zip':'Zip'}, axis = 1, inplace = True)
df_detroit_burbs_2.head()

,suburb_name,Zip,county,latitude,longitude
0,Auburn Hills,48321,Oakland,42.660091,-83.386300
1,Auburn Hills,48326,Oakland,42.661129,-83.245710
2,Berkley,48072,Oakland,42.497035,-83.185320
3,Birmingham,48009,Oakland,42.544084,-83.215270
4,Birmingham,48012,Oakland,42.604425,-83.292382


### Final DataFrame with all data

In [28]:
whole_1 = pd.merge(IDs, ratings_final, how = 'inner', on = 'Venue_ID')
whole_2 = whole_1.replace('NA', np.nan)
whole_2.dropna(inplace = True)
whole_2['Rating'] = whole_2['Rating'].astype(int)
whole_3 = whole_2.groupby('Zip').mean().reset_index()
final = pd.merge(no_pizza, whole_3, on = 'Zip', how = 'inner')
final.sort_values(by = 'Rating', ascending = False, inplace = True)
final.reset_index(drop = True, inplace = True)
final_2 = pd.merge(final, df_detroit_burbs_2, on = 'Zip', how = 'inner')
final_2.drop(['county', 'latitude_y', 'longitude_y'], axis = 1, inplace = True)
final_2.head()

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,latitude_x,longitude_x,Rating,suburb_name
0,48203,Art Gallery,German Restaurant,Yoga Studio,Electronics Store,Food,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,42.421936,-83.099810,8.000000,Highland Park
1,48220,Cocktail Bar,Bar,Massage Studio,Sandwich Place,Mobile Phone Shop,Thai Restaurant,Sushi Restaurant,Gift Shop,Gym,Racetrack,42.460986,-83.133980,7.634615,Ferndale
2,48315,Baby Store,Italian Restaurant,Martial Arts Dojo,Smoke Shop,Other Nightlife,Dry Cleaner,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,42.662902,-82.993920,7.500000,Utica
3,48084,Clothing Store,American Restaurant,Department Store,Accessories Store,Women's Store,Toy / Game Store,Miscellaneous Shop,Boutique,Electronics Store,Coffee Shop,42.563505,-83.184050,7.420000,Troy
4,48037,Pool,Hotel,Community Center,Gym / Fitness Center,Dive Bar,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,42.567543,-83.149978,7.333333,Southfield


### Map 10 Highest Rated Suburbs

In [30]:
map_top_ten = final_2.head(10)

In [31]:
map_detroit_suburbs_final = folium.Map(location = (42.4895, -83.1446), zoom_start = 10)

# add markers to map
for lat, lng, zip1, burb, rating in zip(map_top_ten['latitude_x'], map_top_ten['longitude_x'], map_top_ten['Zip'], map_top_ten['suburb_name'], map_top_ten['Rating']):
    label = '{}, {}, {}'.format(zip1, burb, rating)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#9932cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_detroit_suburbs_final)  
    
map_detroit_suburbs_final

It looks like zip code 48203 (Highland Park) is the best place to open a pizza place. There is no pizza place in the top 10 most common venues and it has the highest average rating among it's other venues.